In [8]:
import cv2
from deepface import DeepFace
from tqdm import tqdm  # Biblioteca para a barra de progresso
import dlib  # Para detecção de landmarks

def process_and_annotate_video(input_video_path, output_video_path):
    # Carregar o vídeo de entrada
    cap = cv2.VideoCapture(input_video_path)
    
    # Obter informações do vídeo de entrada
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Definir o codec e criar o objeto de gravação de vídeo de saída
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Carregar classificadores de rostos frontais e de perfil lateral do OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
    
    # Parâmetros ajustados para reduzir falsos positivos
    scale_factor = 1.2  # Reduz a escala entre as passagens do classificador
    min_neighbors = 6   # Aumenta o número de vizinhos necessários para validar a detecção
    min_size = (50, 50) # Define o tamanho mínimo de rostos a serem detectados
    
    # Definir as emoções consideradas "caretas"
    caretas = ["angry", "disgust", "fear", "happy", "sad", "surprise"]
    contagem_caretas = 0  # Inicializa o contador de caretas
    
    # Inicializar o detector de landmarks de dlib para rostos frontais
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Barra de progresso configurada com o total de frames
    with tqdm(total=total_frames, desc="Processando vídeo", unit="frame") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Converter o frame para escala de cinza para a detecção
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detecção de rostos frontais com hiperparâmetros ajustados
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=scale_factor,
                minNeighbors=min_neighbors,
                minSize=min_size
            )
            
            # Detecção de rostos de perfil lateral
            profiles = profile_cascade.detectMultiScale(
                gray,
                scaleFactor=scale_factor,
                minNeighbors=min_neighbors,
                minSize=min_size
            )
            
            # Processar rostos frontais e adicionar landmarks com dlib
            for (x, y, w, h) in faces:
                # Recortar a região do rosto
                face_region = frame[y:y+h, x:x+w]
                
                try:
                    # Análise de emoções para cada rosto detectado
                    result = DeepFace.analyze(face_region, actions=['emotion'], enforce_detection=False)
                    
                    # Verifique se `result` é uma lista e obtenha o primeiro item, se necessário
                    if isinstance(result, list):
                        result = result[0]
                    
                    # Acessar a emoção dominante
                    emotion = result.get('dominant_emotion', 'Desconhecido')
                    
                    # Verificar se a emoção é uma "careta" e atualizar o contador
                    if emotion in caretas:
                        contagem_caretas += 1
                    
                    # Desenhar a caixa delimitadora no rosto detectado
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    
                    # Adicionar o texto da emoção dominante
                    cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    
                    # Converter as coordenadas do rosto para a estrutura dlib.rectangle
                    dlib_rect = dlib.rectangle(x, y, x + w, y + h)
                    
                    # Detectar landmarks usando dlib para rostos frontais
                    landmarks = predictor(gray, dlib_rect)
                    
                    # Desenhar os landmarks no rosto detectado
                    for i in range(68):
                        x_point = landmarks.part(i).x
                        y_point = landmarks.part(i).y
                        cv2.circle(frame, (x_point, y_point), 2, (0, 255, 0), -1)  # Desenhar ponto com um círculo verde
                
                except Exception as e:
                    print("Erro ao analisar a emoção ou landmarks:", e)
            
            # Processar rostos de perfil
            for (x, y, w, h) in profiles:
                # Desenhar uma caixa delimitadora para rostos laterais
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, "Perfil", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Adicionar o contador de caretas na tela
            cv2.putText(frame, f"Caretas detectadas: {contagem_caretas}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Gravar o frame anotado no vídeo de saída
            out.write(frame)
            
            # Atualizar a barra de progresso
            pbar.update(1)
    
    # Liberação dos objetos de captura e gravação
    cap.release()
    out.release()
    print("Processamento concluído. O vídeo anotado foi salvo em:", output_video_path)
    print("Total de caretas detectadas:", contagem_caretas)

# Caminho do vídeo de entrada e de saída
input_video_path = "Video_tc.mp4"
output_video_path = "SAIDA.avi"

# Executar a função para processar o vídeo
process_and_annotate_video(input_video_path, output_video_path)


Processando vídeo: 100%|██████████| 3326/3326 [05:00<00:00, 11.09frame/s]

Processamento concluído. O vídeo anotado foi salvo em: SAIDA.avi
Total de caretas detectadas: 970
